In [117]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
%config InlineBackend.figure_format = 'svg'

In [118]:
## Load data

def load_data(directory, year):
    file_path = directory + "gl1990-2018/GL" + str(year) + ".TXT"
    currentdf = pd.read_csv(file_path,sep=',',header=None)
    return currentdf

np.arange(1978,2019)
year_range = np.arange(1978, 2019)
directory = './'
for num in year_range:  
    curdf = load_data(directory, num) 
    if num == 1978:
        alldata = curdf
    else:
        alldata = pd.concat([alldata, curdf])

In [119]:
##Only keep Toronto home games
tordata = alldata[alldata[6] == 'TOR']

In [120]:
tordata = tordata.reset_index(drop = True) ## Reset index. Old one is based on all the other teams
tordata[0]=pd.to_datetime(tordata[0],format='%Y%m%d')  ## convert to datetime
tordata.set_index(0, inplace = True) ## set datetime as index

In [121]:
###Extract relevant columns 
data = tordata[[1, 2, 3, 4, 12, 13, 17, 23,51, 9, 10]]
data.columns = ['GameType','DayOfWeek','VisitingTeam_Team','VisitingTeam_League','DayNight','Completion','Attendance','VisitingTeam_Hits','HomeTeam_Hits','VisitingScore','HomeScore']


In [122]:
## Create Winner column based PointDifferential
data['PointDifferential'] = data['HomeScore'] - data['VisitingScore']
data.loc[data['PointDifferential'] > 0, 'Winner'] = 'W'
data.loc[data['PointDifferential'] < 0, 'Winner'] = 'L'

In [123]:
data = data.loc[data['GameType'] != 1] ## Exclude one game of double-header that has no attendance

In [124]:
## Assign a game number for each game of each season
gamesperyear = data.groupby(data.index.year)['GameType'].count().values ## Determine amount of games/year
data['GameNumber'] = ""
for i, year in enumerate(data.index.year.unique().values):
    data.loc[data.index.year == year, 'GameNumber'] = list(range(1,gamesperyear[i]+1))

In [125]:
## WinnerNumber column is useful for calculating WinLossRatio (next cell)
data['WinnerNumber'] = ""
data.loc[data['Winner'] == 'W', 'WinnerNumber'] = 1
data.loc[data['Winner'] == 'L', 'WinnerNumber'] = -1

In [126]:
## Import weather data, set it to index
AllWheather_df = pd.read_csv('./Data/weatherstats_toronto_daily.csv',usecols = ['date','max_temperature','max_relative_humidity','max_wind_speed','avg_visibility','precipitation'])
AllWheather_df['date']=pd.to_datetime(AllWheather_df['date'], format='%y-%m-%d')
AllWheather_df.set_index('date', inplace = True)

In [127]:
data = data.join(AllWheather_df,how='left')

In [128]:
data['year']=data.index.year
data['month']=data.index.month

In [129]:
## Create Homeopener variable
data.loc[data['GameNumber'] == 1, 'HomeOpener'] = 1
data.loc[data['GameNumber'] != 1, 'HomeOpener'] = 0

In [130]:
### Add soccer game information
soccer_df=pd.read_csv('./data/soccer.csv')
soccer_df['Date'] = pd.to_datetime(soccer_df['Date'], format='%Y-%m-%d')
soccer_df.set_index('Date', inplace = True)
soccer_df = soccer_df[soccer_df['Home'] == 'Toronto'] ##only keep toronto home games
soccer_df['Soccergame'] = 1 ## Predictor will be binary yes/no soccer game
soccer_df = soccer_df.drop(['Game #','Home','HomeScore','Visitor','Attendance','VisitorScore'],axis=1) ##drop irrelevant columns

In [131]:
data = data.join(soccer_df, how='left')

In [132]:
data['Soccergame'] = data['Soccergame'].fillna(0)

In [133]:
data.loc[data['DayNight'] == 'D', 'DayNight'] = 0
data.loc[data['DayNight'] == 'N', 'DayNight'] = 1

In [134]:
### For win/loss ratio, load data generated by WinLossRatioForOtherTeams
yankees_df = pd.read_csv('NYA.csv',usecols=['0','NYA_WinLossRatio'])
orioles_df = pd.read_csv('BAL.csv',usecols=['0','BAL_WinLossRatio'])
redsox_df = pd.read_csv('BOS.csv',usecols=['0','BOS_WinLossRatio'])
torontoaway_df = pd.read_csv('TOR.csv',usecols=['0','WinLossRatio','WinLossRatioLast10'])

In [136]:
## Set datetime
def setdatetime(df):
    df['0'] = pd.to_datetime(df['0'],format= '%Y-%m-%d')
    df.set_index('0',inplace=True,drop=True)

setdatetime(yankees_df)
setdatetime(orioles_df)
setdatetime(redsox_df)
setdatetime(torontoaway_df)

In [137]:
## Columns to keep for machine learning
ShortTermPred_df = data[['DayOfWeek','VisitingTeam_Team','DayNight','Attendance','GameNumber','max_temperature','precipitation','year','month','HomeOpener','Soccergame']]
ShortTermPred_df.dropna(inplace=True)

In [ ]:
##Merge win/loss ratio from other teams into main DF
ShortTermPred_df = pd.merge_asof(ShortTermPred_df, yankees_df, left_index=True, right_index=True)
ShortTermPred_df = pd.merge_asof(ShortTermPred_df, orioles_df, left_index=True, right_index=True)
ShortTermPred_df = pd.merge_asof(ShortTermPred_df, redsox_df, left_index=True, right_index=True)
ShortTermPred_df = pd.merge_asof(ShortTermPred_df, torontoaway_df, left_index=True, right_index=True)

In [112]:
## Calculate Gamesback variable
ShortTermPred_df['GamesBack'] = ShortTermPred_df['WinLossRatio'] - ShortTermPred_df[['NYA_WinLossRatio','BAL_WinLossRatio','BOS_WinLossRatio']].max(axis=1) 

In [113]:
## Get holidays column

from datetime import date
import holidays
holidaylist=[]
for date, name in sorted(holidays.CA(state='ON', years=range(1977,2020)).items()):
    holidaylist.append(date)
    
ShortTermPred_df['Holidays'] = ""
ShortTermPred_df.loc[ShortTermPred_df.index.isin(holidaylist), 'Holidays'] = 1
ShortTermPred_df.loc[ShortTermPred_df.index.isin(holidaylist) == False, 'Holidays'] = 0

In [114]:
## Blue Jays moved to new stadium in 1989
ShortTermPred_df.loc[ShortTermPred_df.index.year < 1989,'Stadium'] = 0
ShortTermPred_df.loc[ShortTermPred_df.index.year > 1988,'Stadium'] = 1

In [115]:
ShortTermPred_df.to_csv('ShortTermPred.csv') ## File used for machine learning

In [116]:
###Get historical data for attendance by day of week for web app
HistoricalPerday_df = pd.DataFrame(ShortTermPred_df.groupby('DayOfWeek')['Attendance'].mean().sort_values(ascending=False))
HistoricalPerday_df.index=['Saturday','Sunday','Friday','Thursday','Wednesday','Monday','Tuesday']
HistoricalPerday_df.to_csv('HistoricalAttendanceByDay.csv')